# TensorFlow 2.0

In [1]:
import os
from glob import glob
from datetime import datetime

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 

import matplotlib.pyplot as plt

%load_ext tensorboard
%matplotlib inline

## Hyperparameter Tunning

In [2]:
num_epochs = 5
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Build Model

In [3]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [4]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='sparse_categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

# Data Preprocess

In [5]:
train_paths = glob('../dataset/cifar/train/*.png')[:100]
test_paths = glob('../dataset/cifar/test/*.png')[:100]

In [6]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

In [7]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [8]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot)  # 이번에는 onehot이 아닌 label 번호로

In [9]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32) / 255.  # rescale
    
    label = get_label(path)
    return image, label

In [10]:
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [11]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [13]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

# Checkpoint

In [14]:
save_path = 'ckpt'

In [15]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(save_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

## Training

http://localhost:6006

In [16]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

history = model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs, 
    callbacks=[checkpoint]
)

W0920 13:49:33.031279 22920 deprecation.py:323] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\ops\math_grad.py:1394: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/5
2/3 [===================>..........] - ETA: 0s - loss: 2.3638 - accuracy: 0.0278    
Epoch 00001: val_accuracy improved from -inf to 0.07292, saving model to ckpt


W0920 13:49:34.531265 22920 deprecation.py:506] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\ops\resource_variable_ops.py:1781: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


3/3 [==============================] - 2s 661ms/step - loss: 2.3902 - accuracy: 0.0294 - val_loss: 2.3492 - val_accuracy: 0.0729
Epoch 2/5
2/3 [===================>..........] - ETA: 0s - loss: 2.3568 - accuracy: 0.0938
Epoch 00002: val_accuracy improved from 0.07292 to 0.17708, saving model to ckpt
3/3 [==============================] - 2s 608ms/step - loss: 2.3123 - accuracy: 0.1562 - val_loss: 2.2900 - val_accuracy: 0.1771
Epoch 3/5
1/3 [=========>....................] - ETA: 0s - loss: 2.3461 - accuracy: 0.0625
Epoch 00003: val_accuracy did not improve from 0.17708
3/3 [==============================] - 1s 171ms/step - loss: 2.3539 - accuracy: 0.1471 - val_loss: 2.2858 - val_accuracy: 0.1771
Epoch 4/5
2/3 [===================>..........] - ETA: 0s - loss: 2.3473 - accuracy: 0.0833
Epoch 00004: val_accuracy improved from 0.17708 to 0.19792, saving model to ckpt
3/3 [==============================] - 2s 629ms/step - loss: 2.3158 - accuracy: 0.1471 - val_loss: 2.2940 - val_accuracy: 0

# Saving Model

In [ ]:
save_path = 'my_model.h5'

In [ ]:
model.save(save_path, include_optimizer=True)

In [ ]:
model = tf.keras.models.load_model('my_model.h5')

# Saving Model - 2

In [ ]:
# Save the weights
model.save_weights('model_weights.h5')

# Save the model architecture
with open('model_architecture.json', 'w') as f:
    f.write(model.to_json())

In [ ]:
from tensorflow.keras.models import model_from_json

# Model reconstruction from JSON file
with open('model_architecture.json', 'r') as f:
    model = model_from_json(f.read())

# Load weights into the new model
model.load_weights('model_weights.h5')

# model.h5 들여다보기

In [ ]:
import h5py

model_file = h5py.File('my_model.h5','r+')    

In [ ]:
model_file.keys()

In [ ]:
model_file['model_weights'].keys()

In [ ]:
model_file['model_weights']['conv2d']['conv2d'].keys()

In [ ]:
model_file['model_weights']['conv2d']['conv2d']['kernel:0']

In [ ]:
np.array(model_file['model_weights']['conv2d']['conv2d']['kernel:0'])